In [1]:
import os

import torch
from diffusers import AutoencoderKLWan, WanImageToVideoPipeline, WanTransformer3DModel
from transformers import UMT5EncoderModel, BitsAndBytesConfig
from diffusers.utils import export_to_video, load_image

import numpy as np
from PIL import Image
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from accelerate import Accelerator
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from torchvision.transforms import Compose, ToTensor, Normalize

In [2]:
model_id = "Wan-AI/Wan2.2-TI2V-5B-Diffusers"

vae = AutoencoderKLWan.from_pretrained(model_id, subfolder="vae", torch_dtype=torch.float32)
pipe = WanImageToVideoPipeline.from_pretrained("Wan-AI/Wan2.2-TI2V-5B-Diffusers", torch_dtype=torch.bfloat16)

The config attributes {'clip_output': False} were passed to AutoencoderKLWan, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'clip_output': False} were passed to AutoencoderKLWan, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
pipe.transformer.load_lora_adapter('../lora_fixed/epoch-12.safetensors', prefix=None)

In [4]:
pipe.enable_model_cpu_offload()

## Inference

In [5]:
import pandas as pd
import os

In [6]:
attachment_folder = 'wan_ti2v_5b_12epoch.safetensors'

if not os.path.exists(attachment_folder):
    os.mkdir(attachment_folder)

test_data = pd.read_csv('metadata_test.csv')

In [7]:
for index, row in test_data.loc[:4].iterrows():
    filename = row['video']
    filename = '_'.join(filename.split('_')[:-2]) + '_frame.jpg'
    image = load_image(
        os.path.join("/root/.cache/kagglehub/datasets/davidik67/waves-videos-df/versions/3/first_screen/", filename)
    )

    max_area = 832 * 480
    num_frames = 121
    num_inference_steps = 50
    guidance_scale = 3.5
    
    aspect_ratio = image.height / image.width
    mod_value = pipe.vae_scale_factor_spatial * pipe.transformer.config.patch_size[1]
    height = round(np.sqrt(max_area * aspect_ratio)) // mod_value * mod_value
    width = round(np.sqrt(max_area / aspect_ratio)) // mod_value * mod_value
    image = image.resize((width, height))
    
    prompt = row['prompt']

    output = pipe(
        image=image,
        prompt=prompt,
        height=height,
        width=width,
        num_frames=num_frames,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale
    )
    export_to_video(output.frames[0], os.path.join(attachment_folder, filename.replace('_frame.jpg', '.mp4')), fps=24)

  0%|          | 0/50 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/50 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
